In [1]:
!git clone https://github.com/jeremyng353/ast.git

Cloning into 'ast'...
remote: Enumerating objects: 770, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (209/209), done.
remote: Total 770 (delta 69), reused 239 (delta 56), pack-reused 503
Receiving objects: 100% (770/770), 44.55 MiB | 29.54 MiB/s, done.
Resolving deltas: 100% (329/329), done.


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
%cd ast
!pip install -r requirements.txt

/content/ast
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.3 MB 2.3 MB/s 
     |████████████████████████████████| 10.3 MB 4.0 MB/s 
     |████████████████████████████████| 3.4 MB 57.7 MB/s 
     |████████████████████████████████| 15.3 MB 44.0 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 27.4 MB 2.2 MB/s 
     |████████████████████████████████| 287 kB 12.6 MB/s 
     |████████████████████████████████| 804.1 MB 2.6 kB/s 
     |████████████████████████████████| 1.9 MB 53.3 MB/s 
     |████████████████████████████████| 24.3 MB 83.0 MB/s 
     |████████████████████████████████| 19.1 MB 1.1 MB/s 
     |████████████████████████████████| 19.1 MB 1.2 MB/s 
     |████████████████████████████████| 21.0 MB 1.5 MB/s 
     |████████████████████████████████| 23.2 MB 1.4 MB/s 
     |████████████████████████████████| 23.3 MB 1.5 MB/s 
     |█████████

In [4]:
%cd egs/isolatedurban/

/content/ast/egs/isolatedurban


In [5]:
!rm -r exp

rm: cannot remove 'exp': No such file or directory


In [6]:
# Upload json files before running this block
# %cd egs/audioset/
# TODO: change batch size in run.sh, currently running out of memory in GPU
!python gen_weight_file.py ./data/datafiles/train_data_isolatedurban.json
!bash ./run.sh

usage: gen_weight_file.py [-h] [--data_path DATA_PATH]
gen_weight_file.py: error: unrecognized arguments: ./data/datafiles/train_data_isolatedurban.json
+ source ../../venvast/bin/activate
./run.sh: line 16: ../../venvast/bin/activate: No such file or directory
+ export TORCH_HOME=../../pretrained_models
+ TORCH_HOME=../../pretrained_models
+ model=ast
+ dataset=isolatedurban
+ set=full
+ imagenetpretrain=True
+ '[' full == balanced ']'
+ bal=bal
+ lr=1e-5
+ epoch=5
+ tr_data=./data/datafiles/train_data_isolatedurban.json
+ te_data=./data/datafiles/eval_data_isolatedurban.json
+ freqm=48
+ timem=192
+ mixup=0.5
+ fstride=10
+ tstride=10
+ batch_size=4
+ exp_dir=./exp/test-full-f10-t10-pTrue-b4-lr1e-5-demo
+ '[' -d ./exp/test-full-f10-t10-pTrue-b4-lr1e-5-demo ']'
+ mkdir -p ./exp/test-full-f10-t10-pTrue-b4-lr1e-5-demo
+ CUDA_CACHE_DISABLE=1
+ python -W ignore ../../src/run.py --model ast --dataset isolatedurban --data-train ./data/datafiles/train_data_isolatedurban.json --data-val ./dat

In [7]:
%cd ../../

/content/ast


In [8]:
import sys
from src.models import ASTModel
import os, csv, argparse

# Create a new class that inherits the original ASTModel class
class ASTModelVis(ASTModel):
    def get_att_map(self, block, x):
        qkv = block.attn.qkv
        num_heads = block.attn.num_heads
        scale = block.attn.scale
        B, N, C = x.shape
        qkv = qkv(x).reshape(B, N, 3, num_heads, C // num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)
        attn = (q @ k.transpose(-2, -1)) * scale
        attn = attn.softmax(dim=-1)
        return attn

    def forward_visualization(self, x):
        # expect input x = (batch_size, time_frame_num, frequency_bins), e.g., (12, 1024, 128)
        x = x.unsqueeze(1)
        x = x.transpose(2, 3)

        B = x.shape[0]
        x = self.v.patch_embed(x)
        cls_tokens = self.v.cls_token.expand(B, -1, -1)
        dist_token = self.v.dist_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, dist_token, x), dim=1)
        x = x + self.v.pos_embed
        x = self.v.pos_drop(x)
        # save the attention map of each of 12 Transformer layer
        att_list = []
        for blk in self.v.blocks:
            cur_att = self.get_att_map(blk, x)
            att_list.append(cur_att)
            x = blk(x)
        return att_list

def make_features(wav_name, mel_bins, target_length=1024):
    waveform, sr = torchaudio.load(wav_name)
    assert sr == 16000, 'input audio sampling rate must be 16kHz'

    fbank = torchaudio.compliance.kaldi.fbank(
        waveform, htk_compat=True, sample_frequency=sr, use_energy=False,
        window_type='hanning', num_mel_bins=mel_bins, dither=0.0, frame_shift=10)

    n_frames = fbank.shape[0]

    p = target_length - n_frames
    if p > 0:
        m = torch.nn.ZeroPad2d((0, 0, 0, p))
        fbank = m(fbank)
    elif p < 0:
        fbank = fbank[0:target_length, :]

    fbank = (fbank - (-4.2677393)) / (4.5689974 * 2)
    return fbank


def load_label(label_csv):
    with open(label_csv, 'r') as f:
        reader = csv.reader(f, delimiter=',')
        lines = list(reader)
    labels = []
    ids = []  # Each label has a unique id such as "/m/068hy"
    for i1 in range(1, len(lines)):
        id = lines[i1][1]
        label = lines[i1][2]
        ids.append(id)
        labels.append(label)
    return labels

In [9]:
%cd ./egs/isolatedurban

/content/ast/egs/isolatedurban


In [10]:
import torch, torchaudio, timm
import numpy as np
from torch.cuda.amp import autocast
import IPython

model = ASTModel(label_dim=5, fstride=10, tstride=10, input_fdim=128,
                                  input_tdim=1024, imagenet_pretrain=True,
                                  audioset_pretrain=True, model_size='base384')

# model = ASTModelVis(label_dim=5, input_tdim=1024, imagenet_pretrain=True, audioset_pretrain=True)
checkpoint = torch.load('./exp/test-full-f10-t10-pTrue-b4-lr1e-5-demo/models/best_audio_model.pth', map_location='cuda')
audio_model = torch.nn.DataParallel(model, device_ids=[0])
audio_model.load_state_dict(checkpoint)
audio_model = audio_model.to(torch.device("cuda:0"))
audio_model.eval()

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: True
line 127
line 129
frequncey stride=10, time stride=10
number of patches=1212


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


DataParallel(
  (module): ASTModel(
    (v): DistilledVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): ModuleList(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (act): GELU()
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (drop): Dropout(p=0.0, inplace=False)

In [11]:
!pip install soundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import torch, torchaudio, timm
import librosa
import numpy as np
from torch.cuda.amp import autocast
import IPython
import soundfile as sf

file_path = '/content/E49thAveFrederickSt.wav'

data, sr = librosa.load(file_path, sr=None)
resamp_signal=librosa.resample(data,sr,16000)
signal = resamp_signal;

sf.write(file_path, resamp_signal, 16000, subtype='PCM_24')

# Load the AudioSet label set
label_csv = './data/isolatedurban_label_indices.csv'       # label and indices for audioset data
labels = load_label(label_csv)


feats = make_features(file_path, mel_bins=128)           # shape(1024, 128)
# input_tdim = 1024
input_tdim = feats.shape[0]
print(input_tdim)
feats_data = feats.expand(1, input_tdim, 128)           # reshape the feature
feats_data = feats_data.to(torch.device("cuda:0"))
# do some masking of the input
#feats_data[:, :512, :] = 0.

# Make the prediction
with torch.no_grad():
  with autocast():
    print(f'data: {feats_data}')
    print(f'data shape 0: {feats_data.shape[0]}')
    output = model.forward(feats_data)
    output = torch.sigmoid(output)
result_output = output.data.cpu().numpy()[0]
sorted_indexes = np.argsort(result_output)[::-1]

# Print audio tagging top probabilities
print('Predice results:')
for k in range(5):
    print('- {}: {:.4f}'.format(np.array(labels)[sorted_indexes[k]], result_output[sorted_indexes[k]]))
print('Listen to this sample: ')
IPython.display.Audio(file_path)

1024
data: tensor([[[-0.0353, -0.0285,  0.3484,  ..., -0.3205, -0.5729, -0.9619],
         [ 0.0849,  0.0152,  0.3920,  ..., -0.3456, -0.6095, -1.0266],
         [ 0.1834,  0.0468,  0.4236,  ..., -0.4644, -0.6584, -0.9600],
         ...,
         [-0.2672, -0.4851, -0.1082,  ..., -0.4221, -0.6024, -1.1045],
         [-0.2210, -0.4141, -0.0372,  ..., -0.2422, -0.4881, -1.2775],
         [-0.4119, -0.6119, -0.2350,  ..., -0.4208, -0.6360, -1.0905]]],
       device='cuda:0')
data shape 0: 1
x size: torch.Size([1, 1214, 768])
pos_embed size: torch.Size([1, 1214, 768])
Predice results:
- Engine: 0.8970
- Vehicle horn, car horn, honking: 0.8853
- Speech: 0.4355
- Walk, footsteps: 0.2191
- Emergency vehicle: 0.0433
Listen to this sample: 
